# Capatone Project - Road Users Protection Improvement

## Background
In UK, over 1700 people were killed by traffic accidents every year, and hundreds of thousands injured. These accidents have torn families apart, troubled the communities, and damaged properties. There are many causes to traffic accidents such as bad road using habits or unsafe road design. It is everyone's responsibility to keep the road safe.

The UK Department for Transport revealed that the number of casualties remained broadly consistent in the last 10 years, except for a drop in 2020 due to lockdown for the coronavirus pandemic. The Department also discovered that the number of accidents changed along with traffic volumes.

In this project, we will look into different factors of the road, vehicle and casualty in order to find a pattern of the accidents. Then we can use that information to develop strategies to 
1. reduce the number of accidents
2. protect road users when an accident happens

## Dataset
Datasets:  
- Road Safety Data - Accidents 2019  
- Road Safety Data - Vehicles 2019  
- Road Safety Data - Casualties 2019  

Source: Department for Transport  
Link: https://data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data  
Volume: 117536 accidents, 71 Attributes  
Reliability: Very reliable  
Quality: High  
Data Generation: Info gathered by police  
Ongoing basis: No. New report published in every September

## Table of Contents
1. Import Libraries
2. Load Data
3. Clean Data  
    3.1 Null Values  
    3.2 Invalid Values  
    3.3 SQL  
4. EDA
5. Prediction of Casualty Severity / Casualty Type
6. Identify Casualty Severity Groups by Roads
7. Conclusion

## 1. Import Libraries

In [1]:
import numpy as np
import pandas as pd
import sqlite3

## 2. Load Data

In [2]:
# Choose useful attributes from each dataset
col_accidents = ["Accident_Index", "Accident_Severity", "Number_of_Casualties", "1st_Road_Class", "Road_Type", 
                 "Speed_limit", "Junction_Detail", "Junction_Control", "Pedestrian_Crossing-Physical_Facilities", 
                 "Light_Conditions", "Weather_Conditions", "Road_Surface_Conditions", "Urban_or_Rural_Area"]
col_vehicles = ["Accident_Index", "Vehicle_Type", "Vehicle_Manoeuvre", "Junction_Location"]
col_casualties = ["Accident_Index", "Casualty_Class", "Casualty_Severity", "Pedestrian_Location", "Pedestrian_Movement", 
                  "Casualty_Type"]

# Read .csv
df_accidents = pd.read_csv("data/UK_Road_Safety_Data_2019/Road Safety Data - Accidents 2019.csv", usecols=col_accidents, low_memory=False)
df_vehicles = pd.read_csv("data/UK_Road_Safety_Data_2019/Road Safety Data- Vehicles 2019.csv", usecols=col_vehicles, low_memory=False)
df_casualties = pd.read_csv("data/UK_Road_Safety_Data_2019/Road Safety Data - Casualties 2019.csv", usecols=col_casualties, low_memory=False)

In [3]:
# Show data sample
df_accidents.head(3)

,Accident_Index,Accident_Severity,Number_of_Casualties,1st_Road_Class,Road_Type,Speed_limit,Junction_Detail,Junction_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area
0,2019010128300,3,3,3,1,30,1,2,5,1,1,1,1
1,2019010152270,3,1,3,2,30,0,-1,-1,4,1,1,1
2,2019010155191,3,1,4,6,30,3,4,0,4,1,1,1


In [4]:
df_accidents.tail(3)

,Accident_Index,Accident_Severity,Number_of_Casualties,1st_Road_Class,Road_Type,Speed_limit,Junction_Detail,Junction_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area
117533,2019984107219,3,1,4,6,60,3,4,0,1,1,1,2
117534,2019984107419,3,1,6,6,30,3,4,0,1,1,1,2
117535,201998QC01004,2,1,3,6,60,0,-1,0,1,1,1,2


In [5]:
df_vehicles.head(3)

,Accident_Index,Vehicle_Type,Vehicle_Manoeuvre,Junction_Location
0,2019010128300,9,-1,-1
1,2019010128300,9,-1,-1
2,2019010152270,9,18,0


In [6]:
df_vehicles.tail(3)

,Accident_Index,Vehicle_Type,Vehicle_Manoeuvre,Junction_Location
216378,2019984107219,9,18,1
216379,2019984107419,9,7,6
216380,201998QC01004,5,16,0


In [7]:
df_casualties.head(3)

,Accident_Index,Casualty_Class,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Casualty_Type
0,2019010128300,1,3,0,0,9
1,2019010128300,2,3,0,0,9
2,2019010128300,2,3,0,0,9


In [8]:
df_casualties.tail(3)

,Accident_Index,Casualty_Class,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Casualty_Type
153155,2019984107219,1,3,0,0,9
153156,2019984107419,3,3,5,3,0
153157,201998QC01004,1,2,0,0,5


In [9]:
# Show dataset info
df_accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117536 entries, 0 to 117535
Data columns (total 13 columns):
 #   Column                                   Non-Null Count   Dtype 
---  ------                                   --------------   ----- 
 0   Accident_Index                           117536 non-null  object
 1   Accident_Severity                        117536 non-null  int64 
 2   Number_of_Casualties                     117536 non-null  int64 
 3   1st_Road_Class                           117536 non-null  int64 
 4   Road_Type                                117536 non-null  int64 
 5   Speed_limit                              117536 non-null  int64 
 6   Junction_Detail                          117536 non-null  int64 
 7   Junction_Control                         117536 non-null  int64 
 8   Pedestrian_Crossing-Physical_Facilities  117536 non-null  int64 
 9   Light_Conditions                         117536 non-null  int64 
 10  Weather_Conditions                       117

In [10]:
df_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216381 entries, 0 to 216380
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Accident_Index     216381 non-null  object
 1   Vehicle_Type       216381 non-null  int64 
 2   Vehicle_Manoeuvre  216381 non-null  int64 
 3   Junction_Location  216381 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 6.6+ MB


In [11]:
df_casualties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153158 entries, 0 to 153157
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Accident_Index       153158 non-null  object
 1   Casualty_Class       153158 non-null  int64 
 2   Casualty_Severity    153158 non-null  int64 
 3   Pedestrian_Location  153158 non-null  int64 
 4   Pedestrian_Movement  153158 non-null  int64 
 5   Casualty_Type        153158 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 7.0+ MB


## 3. Clean Data

### 3.1 Null Values

In [12]:
# Count total number of null value in each dataset
print("df_accidents null values: %d" % df_accidents.isnull().sum().sum())
print("df_vehicles null values: %d" % df_vehicles.isnull().sum().sum())
print("df_casualties null values: %d" % df_casualties.isnull().sum().sum())

df_accidents null values: 0
df_vehicles null values: 0
df_casualties null values: 0


In [13]:
# Drop null value if needed
# df_accidents = df_accidents.dropna()
# df_vehicles = df_vehicles.dropna()
# df_casualties = df_casualties.dropna()

### 3.2 Invalid Values

In [14]:
# The value "-1" means missing data or out of range value
# Count the number of "-1" under each column
# If a column doesn't have "-1", do nothing
for col in df_accidents.columns:
    try:
        print(str(col) + ": " + str(df_accidents[col].value_counts()[-1]))
    except:
        pass

print("")
    
for col in df_vehicles.columns:
    try:
        print(str(col) + ": " + str(df_vehicles[col].value_counts()[-1]))
    except:
        pass

print("")

for col in df_casualties.columns:
    try:
        print(str(col) + ": " + str(df_casualties[col].value_counts()[-1]))
    except:
        pass

Accident_Index: 1
Speed_limit: 80
Junction_Detail: 1397
Junction_Control: 52376
Pedestrian_Crossing-Physical_Facilities: 3242
Light_Conditions: 1
Road_Surface_Conditions: 1349

Accident_Index: 1
Vehicle_Type: 309
Vehicle_Manoeuvre: 13063
Junction_Location: 9531

Accident_Index: 1
Pedestrian_Location: 1
Casualty_Type: 5


In [15]:
# Junction Control has too many missing data. Further look into it
df_accidents["Junction_Control"].value_counts()

-1    52376
 4    50882
 2    13122
 3      773
 1      379
 0        4
Name: Junction_Control, dtype: int64

In [16]:
# The top two values of Junction Control are -1) Data missing or out of range, and 
#                                             4) Give way or uncontrolled. 

# Assume all the missing values are "uncontrolled"
df_accidents["Junction_Control"] = np.where(df_accidents["Junction_Control"] == -1, 4, df_accidents["Junction_Control"])

In [17]:
## Find all Accident_Index that contains the value "-1"

dfs = [df_accidents, df_vehicles, df_casualties]
    
# Create empty series to store index
bad_id = pd.Series(dtype="object")
    
# Find the index of rows with "-1" under each column in each dataframe
for df in dfs:
    for col in df.columns:
        ind = df[col][df[col] == -1].index
        # Store "Accident_Index" into "bad_id" according to "ind"
        bad_id = pd.concat([bad_id, df.loc[ind, "Accident_Index"]])
        
# Remove duplication
bad_id = set(bad_id)

print("Number of accidents with missing information: %d" % len(bad_id))
print("Percentage of accidents with missing information: %.2f%%" % (len(bad_id) / len(df_accidents) * 100))

Number of accidents with missing information: 8938
Percentage of accidents with missing information: 7.60%


In [18]:
# Since only less than 10% data has missing information, it is acceptable to delete them
df_accidents = df_accidents.drop(df_accidents[df_accidents["Accident_Index"].isin(bad_id)].index)
df_vehicles = df_vehicles.drop(df_vehicles[df_vehicles["Accident_Index"].isin(bad_id)].index)
df_casualties = df_casualties.drop(df_casualties[df_casualties["Accident_Index"].isin(bad_id)].index)

In [19]:
# Export .csv
df_accidents.to_csv("df.csv", index=False)

In [20]:
# Variable lookup table
lookup = {"Accident_Severity":{1:"Fatal", 
                               2:"Serious", 
                               3:"Slight"}, 
          "1st_Road_Class":{1: "Motorway (High-speed roads)", 
                            2: "A(M) \nA roads with motorway restrictions", 
                            3: "A \nMajor roads", 
                            4: "B \nMinor roads connect different areas, \nfeed traffic to A roads", 
                            5: "C \nSmaller roads connect \nunclassified roads with \nA and B roads", 
                            6: "Unclassified"}, 
          "Road_Type":{1: "Roundabout", 
                       2: "One way street", 
                       3: "Dual carriageway", 
                       6: "Single carriageway", 
                       7: "Slip road", 
                       9: "Unknown", 
                       12: "One way street/Slip road"}, 
          "Junction_Detail":{0: "Not at junction or within 20m", 
                             1: "Roundabout", 
                             2: "Mini-roundabout", 
                             3: "T or staggered junction", 
                             5: "Slip road", 
                             6: "Crossroads", 
                             7: "More than 4 arms \n(not roundabout)", 
                             8: "Private drive or entrance", 
                             9: "Other junction"}, 
          "Junction_Control":{0: "Not at junction or within 20m", 
                              1: "Authorised person", 
                              2: "Auto traffic signal", 
                              3: "Stop sign", 
                              4: "Give way or uncontrolled"}, 
          "Pedestrian_Crossing-Physical_Facilities":{0: "No crossing facilities", 
                                                     1: "Zebra", 
                                                     4: "Pelican, puffin, toucan or similar", 
                                                     5: "Pedestrian phase at traffic signal", 
                                                     7: "Footbridge or subway", 
                                                     8: "Central refuge"}, 
          "Light_Conditions":{1: "Daylight", 
                              4: "Darkness - lights lit", 
                              5: "Darkness - lights unlit", 
                              6: "Darkness - no lighting", 
                              7: "Darkness - lighting unknown"}, 
          "Weather_Conditions":{1: "Fine \nno high winds", 
                                2: "Raining \nno high winds", 
                                3: "Snowing \nno high winds", 
                                4: "Fine \nhigh winds", 
                                5: "Raining \nhigh winds", 
                                6: "Snowing \nhigh winds", 
                                7: "Fog or mist", 
                                8: "Other", 
                                9: "Unknown"}, 
          "Road_Surface_Conditions":{1: "Dry", 
                                     2: "Wet or damp", 
                                     3: "Snow", 
                                     4: "Frost or ice", 
                                     5: "Flood over 3cm. deep", 
                                     6: "Oil or diesel", 
                                     7: "Mud"}, 
          "Urban_or_Rural_Area":{1: "Urban", 
                                 2: "Rural", 
                                 3: "Unallocated"}, 
          "Vehicle_Type":{1: "Pedal cycle", 
                          2: "Motorcycle 50cc and under", 
                          3: "Motorcycle 125cc and under", 
                          4: "Motorcycle over 125cc and up to 500cc", 
                          5: "Motorcycle over 500cc", 
                          8: "Taxi/Private hire car", 
                          9: "Car", 
                          10: "Minibus (8 - 16 passenger seats)", 
                          11: "Bus or coach (17 or more pass seats)", 
                          16: "Ridden horse", 
                          17: "Agricultural vehicle", 
                          18: "Tram", 
                          19: "Van / Goods 3.5 tonnes mgw or under", 
                          20: "Goods over 3.5t. and under 7.5t", 
                          21: "Goods 7.5 tonnes mgw and over", 
                          22: "Mobility scooter", 
                          23: "Electric motorcycle", 
                          90: "Other vehicle", 
                          97: "Motorcycle - unknown cc", 
                          98: "Goods vehicle - unknown weight"}, 
          "Vehicle_Manoeuvre":{1: "Reversing", 
                               2: "Parked", 
                               3: "Waiting to go - held up", 
                               4: "Slowing or stopping", 
                               5: "Moving off", 
                               6: "U-turn", 
                               7: "Turning left", 
                               8: "Waiting to turn left", 
                               9: "Turning right", 
                               10: "Waiting to turn right", 
                               11: "Changing lane to left", 
                               12: "Changing lane to right", 
                               13: "Overtaking moving vehicle - offside", 
                               14: "Overtaking static vehicle - offside", 
                               15: "Overtaking - nearside", 
                               16: "Going ahead left-hand bend", 
                               17: "Going ahead right-hand bend", 
                               18: "Going ahead other"}, 
          "Junction_Location":{0: "Not at or within 20m of junction", 
                               1: "Approaching junction or \nwaiting/parked at junction approach", 
                               2: "Cleared junction or \nwaiting/parked at junction exit", 
                               3: "Leaving roundabout", 
                               4: "Entering roundabout", 
                               5: "Leaving main road", 
                               6: "Entering main road", 
                               7: "Entering from slip road", 
                               8: "Mid Junction \nroundabout or main road"}, 
          "Casualty_Class":{1: "Driver or rider", 
                            2: "Passenger", 
                            3: "Pedestrian"}, 
          "Casualty_Severity":{1: "Fatal", 
                               2: "Serious", 
                               3: "Slight"}, 
          "Pedestrian_Location":{0: "Not a Pedestrian", 
                                 1: "Crossing facility", 
                                 2: "Crossing zig-zag approach lines", 
                                 3: "Crossing zig-zag exit lines", 
                                 4: "Crossing within 50m. of pedestrian crossing", 
                                 5: "In carriageway, crossing elsewhere", 
                                 6: "On footway or verge", 
                                 7: "On refuge, central island or central reservation", 
                                 8: "In centre of carriageway \nnot on refuge, island or central reservation", 
                                 9: "In carriageway, not crossing", 
                                 10: "Unknown or other"}, 
          "Pedestrian_Movement":{0: "Not a Pedestrian", 
                                 1: "Crossing from nearside", 
                                 2: "Crossing from nearside \nmasked by vehicle", 
                                 3: "Crossing from offside", 
                                 4: "Crossing from offside \nmasked by vehicle", 
                                 5: "In carriageway standing or playing", 
                                 6: "In carriageway standing or playing \nmasked by vehicle", 
                                 7: "Walking along in carriageway, \nfacing traffic", 
                                 8: "Walking along in carriageway, \nback to traffic", 
                                 9: "Unknown or other"}, 
          "Casualty_Type":{0: "Pedestrian", 
                           1: "Cyclist", 
                           2: "Motorcycle 050cc and under \nrider or passenger", 
                           3: "Motorcycle 050cc to 125cc \nrider or passenger", 
                           4: "Motorcycle 125cc to 500cc \nrider or  passenger", 
                           5: "Motorcycle 500cc and over \nrider or passenger", 
                           8: "Taxi/Private hire car occupant", 
                           9: "Car occupant", 
                           10: "Bus (8 - 16 passenger seats) occupant", 
                           11: "Bus or coach occupant", 
                           16: "Horse rider", 
                           17: "Agricultural vehicle occupant", 
                           18: "Tram occupant", 
                           19: "Goods vehicle \n(3.5 tonnes mgw and under) occupant", 
                           20: "Goods vehicle \n(3.5 tonnes to 7.5 tonnes mgw) occupant", 
                           21: "Goods vehicle \n(7.5 tonnes mgw and over) occupant", 
                           22: "Scooter rider", 
                           23: "Electric motorcycle rider or passenger", 
                           90: "Other vehicle occupant", 
                           97: "Motorcycle unknown cc \nrider or passenger", 
                           98: "Goods vehicle \n(unknown weight) occupant"}}

In [21]:
%%time

# Convert codes to descriptions
for col in df_accidents.columns:
    try:
        df_accidents[col] = df_accidents[col].map(lookup[col])
    except:
        pass

for col in df_vehicles.columns:
    try:
        df_vehicles[col] = df_vehicles[col].map(lookup[col])
    except:
        pass
    
for col in df_casualties.columns:
    try:
        df_casualties[col] = df_casualties[col].map(lookup[col])
    except:
        pass

Wall time: 125 ms


### 3.3 SQL

In [22]:
## Create SQL database for query

# Create/Connect database
conn = sqlite3.connect("uk_road_2019.sqlite")
# Create a cursor
c = conn.cursor()

In [23]:
# Drop previous work
c.execute("DROP TABLE IF EXISTS accidents;")
c.execute("DROP TABLE IF EXISTS vehicles;")
c.execute("DROP TABLE IF EXISTS casualties;")
conn.commit()

In [24]:
# Copy dataframe to SQL table
df_accidents.to_sql("acc", con=conn, if_exists="replace", index=False)
df_vehicles.to_sql("veh", con=conn, if_exists="replace", index=False)
df_casualties.to_sql("cas", con=conn, if_exists="replace", index=False)

In [25]:
# df.to_sql() function doesn't create primary keys, so we need to work around it by creating new tables
c.executescript("""

CREATE TABLE accidents (id varchar(255) NOT NULL PRIMARY KEY, 
                        accident_severity varchar(255), 
                        no_of_casualties varchar(255), 
                        road_class varchar(255), 
                        road_type varchar(255), 
                        speed_limit varchar(255), 
                        junction_detail varchar(255), 
                        junction_control varchar(255), 
                        pedestrian_crossing_facilities varchar(255), 
                        light_condition varchar(255), 
                        weather_condition varchar(255), 
                        road_surface_condition varchar(255), 
                        urban_or_rural varchar(255));
INSERT INTO accidents SELECT * FROM acc;
DROP TABLE acc;

CREATE TABLE vehicles (id varchar(255) NOT NULL, 
                       vehicle_type varchar(255), 
                       vehicle_manoeuvre varchar(255), 
                       junction_location varchar(255), 
                       FOREIGN KEY (id) REFERENCES accidents (id));
INSERT INTO vehicles SELECT * FROM veh;
DROP TABLE veh;

CREATE TABLE casualties (id varchar(255) NOT NULL, 
                         casualty_class varchar(255), 
                         casualty_severity varchar(255), 
                         pedestrian_location varchar(255), 
                         pedestrian_movement varchar(255), 
                         casualty_type varchar(255), 
                         FOREIGN KEY (id) REFERENCES accidents (id));
INSERT INTO casualties SELECT * FROM cas;
DROP TABLE cas;

""")

## 4. EDA
See Capstone_EDA.ipynb
## 5. Prediction of Casualty Severity / Casualty Type
See Capstone_Modelling.ipynb
## 6. Identify Casualty Severity Groups by Roads
See Capstone_Modelling.ipynb
## 7. Conclusion
See Report.docx